<a href="https://colab.research.google.com/github/yleessam/tf/blob/main/MLP_XOR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# numpy 임포트

In [ ]:
import numpy as np

# 다층 퍼셉트론 클래스

In [ ]:
class NeuralNetwork:
    def __init__(self, layers, alpha=0.1):
        # 생성자: 신경망 초기화
        # layers: 각 레이어에 있는 노드의 개수를 나타내는 리스트
        # :(예: [2, 2, 1]은 입력 레이어에 2개, 은닉 레이어에 2개,
        # : 출력 레이어에 1개의 노드를 갖는 신경망을 의미)
        # alpha: 학습률
        self.W = [] # 가중치를 저장할 리스트
        self.layers = layers  # list [ 2, 2, 2, 1]
        self.alpha = alpha

        # 가중치 초기화: 입력 레이어부터 마지막 은닉 레이어까지의 가중치를 초기화
        # 층을 반복하면서 각 층의 노드 수와 다음 층의 노드 수를 기반으로 가중치 매트릭스를 생성합니다.
        # 가중치는 표준 정규 분포를 사용해 초기화하며, He 초기화 방법을 사용하여 가중치를 스케일링합니다.
        for i in np.arange(0, len(layers) - 2):
            # len(layer) -2 -> out -마지막 hidden layer 제외
            w = np.random.randn(layers[i]+1 ,layers[i+1]+1)
            self.W.append(w/np.sqrt(layers[i]))

        # 마지막 은닉층에서 출력층으로의 가중치를 초기화합니다., 마지막 hidden - out 사이의 계수
        w = np.random.randn(layers[-2]+1, layers[-1])
        self.W.append(w/np.sqrt(layers[-2]))

    def sigmoid(self, x):
        # 시그모이드 활성화 함수 식을 파이썬으로 표현한 것: 신경망의 각 노드에서 사용
        #시그모이드 함수는 일반적으로 로지스틱 함수로도 알려져 있으며, 실수 입력값을 받아서 (0, 1) 범위의 출력값을 반환

        return 1.0 / (1 + np.exp(-x))

    def sigmoid_deriv(self, x):
        # 시그모이드 활성화 함수의 미분식: 이는 역전파 시 그래디언트(기울기)를 계산할 때 사용

        return x * (1 - x)

    # 신경망을 학습시키는 메서드입니다.
    # 입력 X = np.array([[ 0,0], [ 0,1], [1,0], [0,0]])
    # 타겟 y = np.array([[0], [1], [1], [0]])
    # epochs: 학습을 위해 데이터셋을 반복할 횟수입니다.
    def fit(self, X, y, epochs=1000):
        # 입력 데이터에 바이어스를 위한 1 추가, 4x2 행렬에 바이어스를 넣어서 4x3 행렬

        X = np.c_[X, np.ones((X.shape[0]))] # X.shape[0] = 4, 레이어수4
        #print(X)

        for epoch in np.arange(0, epochs):
            for (x, target) in zip(X, y):
                # x = [0 0 1] target = [0]
                #print(x, target)

                # 각 데이터 샘플에 대해 부분 학습 수행
                self.fit_partial(x, target)

    def fit_partial(self, x, y):
        # 부분 학습 메서드: x는 단일 데이터 샘플, y는 해당 타겟 레이블
        # x = [ 0 0 1] y = [0]
        A = [np.atleast_2d(x)] # 입력 데이터를 2차원 배열로 변환

        # 순전파 단계: 입력 데이터가 네트워크를 통해 전파되면서 각 레이어의 출력 계산
        for layers in np.arange(0, len(self.W)):
            net = A[layers].dot(self.W[layers])#각 노드의 값 * 가중치
            out = self.sigmoid(net) #활성화함수 처리
            A.append(out)

        # 역전파 단계: 신경망의 출력과 실제 타겟 값의 차이를 계산합니다.

        # 오류 계산: 신경망의 최종 출력과 타겟 레이블과의 차이
        # A= [array([[0., 0., 1.]]), array([[0.54067744, 0.51712677, 0.26921834]]), array([[0.50808391, 0.57745501, 0.56186676]]), array([[0.4979378]])]
        error = A[-1] - y

        # 오류의 시그모이드 미분값을 사용하여 오류 신호 계산
        # 이는 출력층에서의 그래디언트입니다.
        D = [error * self.sigmoid_deriv(A[-1])]

        # 모든 층에 대해 역전파를 수행합니다. 출력층부터 시작하여 입력층 방향으로 진행합니다.
        for layers in np.arange(len(A) -2, 0, -1):
            # 오류 신호를 가중치의 전치와 곱합니다.
            delta = D[-1].dot(self.W[layers].T)

            # 은닉층에서의 그래디언트는 현재 층의 활성화 함수 미분과 delta를 곱하여 계산합니다.
            delta = delta*self.sigmoid_deriv(A[layers])
            D.append(delta)
        #print(D)

        # D 리스트를 뒤집습니다. 이렇게 하여 입력층에 가까운 오류부터 시작하게 됩니다.
        D = D[::-1]

        # 가중치 업데이트: 모든 레이어에 대해 가중치를 업데이트합니다.
        for layer in np.arange(0, len(self.W)):
            # 현재 레이어의 출력과 오류 신호를 곱하여 가중치 업데이트에 대한 기여도를 계산합니다.
            # 학습률 alpha를 곱하여 가중치 업데이트를 조정합니다.
            self.W[layer] += -self.alpha * A[layer].T.dot(D[layer])

    # 예측 메서드: 새로운 데이터에 대한 예측을 수행합니다.
    def predict(self, X):  # X = [0 1]
        p = np.atleast_2d(X)# 입력 데이터를 최소 2차원 배열로 변환합니다.
        p = np.c_[p, np.ones((p.shape[0]))]# 바이어스를 위한 1을 추가합니다.

        # 신경망을 통해 순전파를 수행하여 예측 결과를 계산합니다.
        for layer in np.arange(0, len(self.W)):
            p = self.sigmoid(np.dot(p, self.W[layer]))
        return p

# 다층 퍼셉트론 클래스 객체 생성,

1 hidden layer, learning rate 0.5

In [ ]:
nn = NeuralNetwork([2, 2, 1], 0.5)

# XOR 입력 데이터 정의, 출력값 정의, 트레이닝(back-propagation)

In [ ]:
X = np.array([[ 0,0], [ 0,1], [1,0], [1,1]])
y = np.array([[0], [1], [1], [0]])
nn.fit(X, y, 1000)

# feed-forward 예측

In [ ]:
for (x, target) in zip(X, y):
    pred = nn.predict(x)[0][0]
    step = 1 if pred > 0.5 else 0
    print(target[0], pred, step)

0 0.1286980319860896 0
1 0.6397531690352667 1
1 0.6368536285734462 1
0 0.637384855442132 1
